In [11]:
%autosave 20

import pickle
from functools import partial
from multiprocessing import cpu_count
from multiprocessing import Pool as ProcessPool
from multiprocessing.pool import ThreadPool
from threading import Thread
from queue import Queue

import numpy as np

Autosaving every 20 seconds


In [2]:
cpus = cpu_count()
repeat = 8*cpus

In [3]:
A = np.random.rand(64, 64, 5, 5)
def f(x):
    return np.linalg.eigvals(A + x)

# def f(x):
#     return np.log(np.random.rand(100_000)**2 + x*500 + 1)**np.pi

# A = np.random.rand(100_000, 2)
# b = np.random.rand(100_000)
# def f(x):
#     return np.linalg.lstsq(A, b, rcond=None)

# a = list(range(1_000_000))

# def f(x, y):
#     return sum(a) * x + y

In [4]:
%timeit f(0)

41 ms ± 1.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%%timeit
for i in range(repeat):
    f(i)

2.34 s ± 5.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit
threads = tuple(Thread(target=f, daemon=False, args=(i,))
                for i in range(repeat))
for t in threads:
    t.start()
for t in threads:
    t.join()

331 ms ± 7.87 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
with ThreadPool(processes=cpus) as pool:
    result = pool.map(f, range(repeat))

389 ms ± 34.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
with ProcessPool(processes=cpus) as pool:
#     result = pool.map(partial(f, 2), range(repeat))
    result = pool.map(f, range(repeat))

437 ms ± 1.65 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
b = pickle.dumps([1,2])
pickle.loads(b)

[1, 2]

In [16]:
q = Queue()
q.put(1)
q.get(timeout=1)
q.get()

KeyboardInterrupt: 